In [2]:
import pandas as pd
import gzip
import numpy as np
import json

In [3]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Pet_Supplies_5.json.gz')

In [4]:
columns = ['reviewerID','reviewerName','unixReviewTime','reviewTime','asin','helpful']
df = df.drop(columns, axis=1)

In [5]:
df.overall.describe()
#Mean rating was 4.23, lowest rating was 1 and half of the ratings were 5/5.

count    157836.000000
mean          4.229745
std           1.175819
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: overall, dtype: float64

In [6]:
#Merge Summary and reviewText to form text
df['text'] = df['reviewText'].astype(str) + df['summary']

In [7]:
positive_keywords = ['great','enjoyed','loves','like','recommend',
                     'happy','yummy','nice','very','good','easy','useful','awesome','works','love']
negative_keywords = ['not','disgusting','junk','bad','didn\'t work','waste','crap']

for key in positive_keywords:
    df[str(key)] = df.text.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [8]:
df['sentiment'] = np.where(df['text'].str.contains
                           ('(great|enjoyed|enjoys|loves|like|happy|yummy|nice|very|good|easy|useful|awesome|works|love|recommend|worth)',
                                        regex=True,case=False), 1, 0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
positive_data = df[positive_keywords]
df['sentiment'] = (df['sentiment'] == 1)
target = df['sentiment']

In [10]:
#import the Bernoulli classifier since data is binary/boolean
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(positive_data, target)

#Classify storing the result in a new variable.
y_pred = bnb.predict(positive_data)

print("Number of Negative statements out of a total of {} points : {}".format(
     positive_data.shape[0],
     (target != y_pred).sum()
))

Number of Negative statements out of a total of 157836 points : 15863


In [15]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
X = positive_data
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(126268, 15) (126268,)
(31568, 15) (31568,)


In [16]:
from sklearn.naive_bayes import BernoulliNB
#Cross validation model
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits = 5, shuffle=True, random_state = 40)
bnb = BernoulliNB() 
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    bnb = bnb.fit(X_train, y_train)
    print("Score:", bnb.score(X_test, y_test))

Score: 0.897364419665
Score: 0.899673709887
Score: 0.898628314379
Score: 0.901162606519
Score: 0.900655748091
